# UCI MTB DH Data Retrieval

## Setup
#### Import Libraries

If you do not have these libraries available, you should install them using `pip`

```
pip install requests
pip install bs4
pip install pandas
```

In [67]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime as dt
import os

In [68]:
def calculate_age(born):
    today = dt.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

Widen display area to prevent column wrapping, and always show all columns for debug 

In [69]:
pd.set_option('display.width', 2000)
pd.set_option('display.max_columns', None)

## Config

Which race data are we collecting?

1. Losinj
1. Fort William
1. Leogang
1. Val di Sole
1. Vallnord
1. Mont-Sainte-Anne
1. La Bresse

In [70]:
race = 2
gender = 'f'
event = 'dh'
# quali = False

#### Data Sources

The UCI Live Timing API contains a lot of data points, but not all the ones we want (speed being the main one missing), and not even all the ones they include on their own PDF which is frustrating.

Similarly, Roots & Rain also has a lot of the data points, but again not all of them; most notably it's missing timing splits 4 and 5.

Therefore we need to pull from both sources and combine the sets.

We specify the URLs for both sources from which we will extract our data. The UCI API URL can be found by loading the Live Timing page then using your browser's inspector on the Network tab (in Chrome at least) to see the data feed. As the UCI seems to be using a Single Page Application (SPA) here, it's not straight forward to extract this link automagically.

**Note:** The Race list is now maintained as an external python config file `config.py` imported below

In [94]:
from config import races

racename = races[race]['name']
urlUci = races[race]['urls']['uci'] + str(( 3 if 'm' == gender else 6 )) + '/'
urlUciQ = races[race]['urls']['uci'] + str(( 2 if 'm' == gender else 5 )) + '/'
urlRoots = races[race]['urls']['rnr'] + gender + '/'

In [95]:
display( urlUci, urlUciQ, urlRoots )

'http://prod.chronorace.be/api/results/uci/dh/race/20180602_dh/6/'

'http://prod.chronorace.be/api/results/uci/dh/race/20180602_dh/5/'

'https://www.rootsandrain.com/race5898/2018-jun-3-mercedes-benz-uci-world-cup-2-fort-william/results/filters/f/'

File handling setup

In [72]:
directory = event + str(race) + '_' + racename
if not os.path.exists(directory):
    os.makedirs(directory)

file_prefix = event + str(race) + '_' + racename + '_' + gender
file_prefix = os.path.join( directory, file_prefix )

# UCI API
### Load Data

These two lines make the actual request to the server, and then converts the JSON string response in to a usable list format (deserialization)

In [73]:
d = requests.get( urlUci ).json()
q = requests.get( urlUciQ ).json()

The API returns with three main sections:

1. `Last Finisher`
 - Racers in order of start time
2. `Results`
 - Racers in finishing rank order
3. `Riders`
 - Personal details on all racers
 
Each contains many data points. To see all the contained data, you can un-comment and execute any of the lines in the next section to explore more.

In [74]:
# display( d )
# display( d['Results'][7] )
# display( d['Riders']['1001'] )
# display( d['Results'][61] )

### Extract Data

Here we iterate over the `Results` sub-set of data to extract the information we care about: basically those that finished the race, some identifying info, and their splits.

If you looked at detail of the returned data set in the last step you might have noticed the rider's name is not stored next to their result, riders are only identified by a reference number. To facilitate our analysis later on it is useful to import each rider's name at this stage by cross-referencing the `Riders` sub-set.

First we find out the last man to drop-in's start number so we can use that to add a reverse order column.

In [75]:
lastStart = d['Riders'][list(d['Riders'].keys())[-1]]['StartOrder']

We start with an empty list `lst` and in each loop iteration add an entry (actually a dict) to that list for each rider.

In [76]:
splits = len(d['Results'][0]['Times'] )
lst = []
for idx, row in enumerate( d['Results'] ):
    fin = "Finished" == row['Status']
    res = {
        'rank': row['Position'] if fin else idx+1,
        'name': d['Riders'][str(row['RaceNr'])]['PrintName'],
        'id': row['RaceNr'],
        'uci': d['Riders'][str(row['RaceNr'])]['UciRiderId'],
        'bib': d['Riders'][str(row['RaceNr'])]['RaceNr'],
        'status': row['Status'],
        'speed': np.nan,
        'start': d['Riders'][str(row['RaceNr'])]['StartOrder'],
        'start_rev': lastStart - d['Riders'][str(row['RaceNr'])]['StartOrder'] +1
    }

    # Add all splits to result set
    for split in range( 0, splits ):
        res['split' + str(split+1)] = row['Times'][split]['RaceTime']/1000 if fin else np.nan

    # Append result set to list
    lst.append(res)

This line loads the completed list in to a Pandas dataframe so that we can easily write it out to CSV later on 

In [77]:
df = pd.DataFrame( lst )

#### Expand Dataset

Calculate and add all the extra columns we need for split and sector differences and their rankings

In [78]:
for i in range( 1, splits+1 ):
    split = 'split' + str(i)
    sector = split + '_sector'
    df[split + '_rank'] = df[split].rank(method='dense')
    df[split + '_vs_best'] = (df[split] - df[split].min())
    df[split + '_vs_winner'] = (df[split] - df[split][0])

    if i > 1:
        df[split + '_sector'] = df[split] - df['split' + str(i-1)]
        df[split + '_sector_rank'] = df[sector].rank(method='dense')
        df[split + '_sector_vs_best'] = (df[sector] - df[sector].min())
        df[split + '_sector_vs_winner'] = (df[sector] - df[sector][0])

We can take a peek at our data at this point to make sure it looks how we expect.

At this point the `speed` column is NaN (Not a Number) for all racers. This will be filled in below.

In [79]:
display( df['split1'][0], df['split1'].min() )

64.813999999999993

64.813999999999993

In [80]:
display( df.head(10) )

,bib,id,name,rank,speed,split1,split2,split3,split4,split5,start,start_rev,status,uci,split1_rank,split1_vs_best,split1_vs_winner,split2_rank,split2_vs_best,split2_vs_winner,split2_sector,split2_sector_rank,split2_sector_vs_best,split2_sector_vs_winner,split3_rank,split3_vs_best,split3_vs_winner,split3_sector,split3_sector_rank,split3_sector_vs_best,split3_sector_vs_winner,split4_rank,split4_vs_best,split4_vs_winner,split4_sector,split4_sector_rank,split4_sector_vs_best,split4_sector_vs_winner,split5_rank,split5_vs_best,split5_vs_winner,split5_sector,split5_sector_rank,split5_sector_vs_best,split5_sector_vs_winner
0,3,2003,SEAGRAVE Tahnee,1,NaN,64.814,211.338,240.134,281.291,310.960,14,3,Finished,10007414016,1.0,0.000,0.000,1.0,0.000,0.000,146.524,1.0,0.000,0.000,1.0,0.000,0.000,28.796,1.0,0.000,0.000,1.0,0.000,0.000,41.157,2.0,0.034,0.000,1.0,0.000,0.000,29.669,1.0,0.000,0.000
1,1,2001,NICOLE Myriam,2,NaN,65.404,216.908,247.122,288.983,319.211,15,2,Finished,10004535237,3.0,0.590,0.590,2.0,5.570,5.570,151.504,5.0,4.980,4.980,2.0,6.988,6.988,30.214,4.0,1.418,1.418,2.0,7.692,7.692,41.861,4.0,0.738,0.704,2.0,8.251,8.251,30.228,7.0,0.559,0.559
2,2,2002,ATHERTON Rachel,3,NaN,65.120,218.276,247.811,289.281,320.016,16,1,Finished,10003434487,2.0,0.306,0.306,3.0,6.938,6.938,153.156,8.0,6.632,6.632,3.0,7.677,7.677,29.535,2.0,0.739,0.739,3.0,7.990,7.990,41.470,3.0,0.347,0.313,3.0,9.056,9.056,30.735,11.0,1.066,1.066
3,4,2004,CABIROU Marine,4,NaN,68.834,219.147,249.423,291.659,321.782,11,6,Finished,10009563069,5.0,4.020,4.020,4.0,7.809,7.809,150.313,2.0,3.789,3.789,4.0,9.289,9.289,30.276,5.0,1.480,1.480,4.0,10.368,10.368,42.236,6.0,1.113,1.079,4.0,10.822,10.822,30.123,5.0,0.454,0.454
4,7,2007,HANNAH Tracey,5,NaN,69.640,221.078,251.784,292.907,322.744,13,4,Finished,10003732258,8.0,4.826,4.826,6.0,9.740,9.740,151.438,4.0,4.914,4.914,6.0,11.650,11.650,30.706,7.0,1.910,1.910,5.0,11.616,11.616,41.123,1.0,0.000,-0.034,5.0,11.784,11.784,29.837,2.0,0.168,0.168
5,5,2005,RAVANEL Cecile,6,NaN,69.940,220.347,250.502,293.668,324.368,12,5,Finished,10002816317,10.0,5.126,5.126,5.0,9.009,9.009,150.407,3.0,3.883,3.883,5.0,10.368,10.368,30.155,3.0,1.359,1.359,6.0,12.377,12.377,43.166,11.0,2.043,2.009,6.0,13.408,13.408,30.700,10.0,1.031,1.031
6,9,2009,CURD Katy,7,NaN,68.445,222.227,252.625,295.056,325.459,7,10,Finished,10006044292,4.0,3.631,3.631,9.0,10.889,10.889,153.782,9.0,7.258,7.258,7.0,12.491,12.491,30.398,6.0,1.602,1.602,7.0,13.765,13.765,42.431,7.0,1.308,1.274,7.0,14.499,14.499,30.403,9.0,0.734,0.734
7,8,2008,SIEGENTHALER Emilie,8,NaN,69.004,221.473,252.675,295.470,326.604,9,8,Finished,10004167243,6.0,4.190,4.190,7.0,10.135,10.135,152.469,6.0,5.945,5.945,8.0,12.541,12.541,31.202,10.0,2.406,2.406,8.0,14.179,14.179,42.795,9.0,1.672,1.638,8.0,15.644,15.644,31.134,12.0,1.465,1.465
8,10,2010,SALAZAR Mariana,9,NaN,69.152,221.653,253.741,296.977,328.307,8,9,Finished,10007687838,7.0,4.338,4.338,8.0,10.315,10.315,152.501,7.0,5.977,5.977,9.0,13.607,13.607,32.088,13.0,3.292,3.292,9.0,15.686,15.686,43.236,13.0,2.113,2.079,9.0,17.347,17.347,31.330,13.0,1.661,1.661
9,14,2014,WIDMANN Veronika,10,NaN,70.073,224.835,255.885,299.410,331.650,5,12,Finished,10007423312,12.0,5.259,5.259,10.0,13.497,13.497,154.762,10.0,8.238,8.238,10.0,15.751,15.751,31.050,9.0,2.254,2.254,10.0,18.119,18.119,43.525,15.0,2.402,2.368,10.0,20.690,20.690,32.240,15.0,2.571,2.571


#### Rider Data

Saving the personal information about each racer is much easier as we can just export the entire `Riders` dataset. However, the rows and columns are the wrong way round so the `.T` command *transposes* the information, meaning it basically flips the axes.

In [81]:
df2 = pd.DataFrame( d['Riders'] )
df2 = df2.T
df2['Age'] = [ calculate_age( dt.datetime.strptime( dob[:10], "%Y-%m-%d" ) ) for dob in df2['BirthDate'] ]

Here we can glimpse the first few rows of our `DataFrame` and can check the data looks as we expect

In [82]:
display( df2.head() )

,BirthDate,CategoryCode,FamilyName,GivenName,Id,Nation,Outfit,PrintName,RaceId,RaceNr,ScoreboardName,StartOrder,StartTime,UciCode,UciRank,UciRiderId,UciTeamCode,UciTeamId,UciTeamName,WorldCupRank,Age
2001,1990-02-08T00:00:00,WE,NICOLE,Myriam,102001,FRA,WCL,NICOLE Myriam,0,1,NICOLE M,15,47040000,FRA19900208,1,10004535237,CVN,1590,COMMENCAL / VALLNORD,1,28
2002,1987-12-06T00:00:00,WE,ATHERTON,Rachel,102002,GBR,NCh,ATHERTON Rachel,0,2,ATHERTON R,16,47280000,GBR19871206,6,10003434487,TDH,1598,TREK FACTORY RACING DH,2,30
2003,1995-06-15T00:00:00,WE,SEAGRAVE,Tahnee,102003,GBR,NaN,SEAGRAVE Tahnee,0,3,SEAGRAVE T,14,46800000,GBR19950615,2,10007414016,FMD,1863,TRANSITION BIKES / MUC-OFF FACTORY RACING,3,22
2004,1997-03-12T00:00:00,WE,CABIROU,Marine,102004,FRA,NaN,CABIROU Marine,0,4,CABIROU M,11,46140000,FRA19970312,4,10009563069,VVR,2096,VOULVOUL RACING,4,21
2005,1981-01-06T00:00:00,WE,RAVANEL,Cecile,102005,FRA,NaN,RAVANEL Cecile,0,5,RAVANEL C,12,46350000,FRA19810106,38,10002816317,,0,,5,37


# Roots and Rain
### Load Data

Similar to the UCI api, we make a request to the server with the previously declared `urlRoots` variable. This time however we simply load the content of the response as text which is actually the HTML code of the web page. We do not do have a nice JSON API to read which means we will not deserialize.

Next we invoke a utility called `BeautifulSoup` to help us extract the data from this messy HTML code

In [83]:
r = requests.post( urlRoots )
c = r.content
soup = BeautifulSoup( c, "html.parser" )

### Extract Data

If you look at the Roots and Rain page you'll see it listed in a tabular format. What we do here is find all the rows of that table so we can extract the information we need.

Specifically we are looking for instances of `tr` (table row), with a class that *begins with* `c-` as this is a common denomenator I discovered when looking through the code with the browser inspector

In [84]:
rows = soup.find_all( "tr", class_=lambda x: x and 'c-' in x )

Similar to the UCI data set, here we will iterate over each row in our data set--basically each table row from the web page--and extract the bits we need.

Racer speed is the metric we're interested in, but in order to match that to our existing data set we need a corresponding identifier so we also extract the racer licence number as that exists in both sets and we can match them together: it is the *intersect* between both sets of data.

To summarise:
1. Extract licence number and corresponding speed
2. Import speed to existing DataFrame matching racers by licence

The `if` condition in the middle will exit this block of code once we hit the end of the Elite finishers, seeing as that's all we have in our existing data set so can't match anyone else

In [91]:
for row in rows:
    cells = row.find_all( "td" )
    spd = cells[12 if False == quali else 7].text[:5]
    speed = float( spd if 0 < len(spd) else 0 )
    licence = cells[4].text
    bib = int( cells[1].text )
    pos = cells[0].text[8:]
    if "" == pos: break

    # Match rider by UCI licence if present, otherwise fallback to bib
    if len(df2.loc[df2['UciRiderId'] == licence].index.values ):
        rid = int(df2.loc[df2['UciRiderId'] == licence].index.values[0])
    else:
        rid = int( df2.loc[df2['RaceNr'] == bib].index.values[0] )

    # Add speed, and other associated metrics
    df.loc[df['id'] == rid, 'speed'] = speed
    df.loc[df['id'] == rid, 'speed_ms'] = float(speed)*(1000/60/60)
    df.loc[df['id'] == rid, 'speed_ms_vs_best'] = df['speed_ms'].max() - df.speed_ms
    df['speed_rank'] = df.speed.rank(method='dense', ascending=False)

As before, we can take another look at how our data is looking, with the `speed` column now containing data 

In [86]:
display( df.head() )

,bib,id,name,rank,speed,split1,split2,split3,split4,split5,start,start_rev,status,uci,split1_rank,split1_vs_best,split1_vs_winner,split2_rank,split2_vs_best,split2_vs_winner,split2_sector,split2_sector_rank,split2_sector_vs_best,split2_sector_vs_winner,split3_rank,split3_vs_best,split3_vs_winner,split3_sector,split3_sector_rank,split3_sector_vs_best,split3_sector_vs_winner,split4_rank,split4_vs_best,split4_vs_winner,split4_sector,split4_sector_rank,split4_sector_vs_best,split4_sector_vs_winner,split5_rank,split5_vs_best,split5_vs_winner,split5_sector,split5_sector_rank,split5_sector_vs_best,split5_sector_vs_winner,speed_ms,speed_ms_vs_best,speed_rank
0,3,2003,SEAGRAVE Tahnee,1,52.20,64.814,211.338,240.134,281.291,310.960,14,3,Finished,10007414016,1.0,0.000,0.000,1.0,0.000,0.000,146.524,1.0,0.000,0.000,1.0,0.000,0.000,28.796,1.0,0.000,0.000,1.0,0.000,0.000,41.157,2.0,0.034,0.000,1.0,0.000,0.000,29.669,1.0,0.000,0.000,14.500000,0.000000,3.0
1,1,2001,NICOLE Myriam,2,50.72,65.404,216.908,247.122,288.983,319.211,15,2,Finished,10004535237,3.0,0.590,0.590,2.0,5.570,5.570,151.504,5.0,4.980,4.980,2.0,6.988,6.988,30.214,4.0,1.418,1.418,2.0,7.692,7.692,41.861,4.0,0.738,0.704,2.0,8.251,8.251,30.228,7.0,0.559,0.559,14.088889,0.411111,8.0
2,2,2002,ATHERTON Rachel,3,49.81,65.120,218.276,247.811,289.281,320.016,16,1,Finished,10003434487,2.0,0.306,0.306,3.0,6.938,6.938,153.156,8.0,6.632,6.632,3.0,7.677,7.677,29.535,2.0,0.739,0.739,3.0,7.990,7.990,41.470,3.0,0.347,0.313,3.0,9.056,9.056,30.735,11.0,1.066,1.066,13.836111,0.663889,11.0
3,4,2004,CABIROU Marine,4,50.55,68.834,219.147,249.423,291.659,321.782,11,6,Finished,10009563069,5.0,4.020,4.020,4.0,7.809,7.809,150.313,2.0,3.789,3.789,4.0,9.289,9.289,30.276,5.0,1.480,1.480,4.0,10.368,10.368,42.236,6.0,1.113,1.079,4.0,10.822,10.822,30.123,5.0,0.454,0.454,14.041667,0.458333,9.0
4,7,2007,HANNAH Tracey,5,53.08,69.640,221.078,251.784,292.907,322.744,13,4,Finished,10003732258,8.0,4.826,4.826,6.0,9.740,9.740,151.438,4.0,4.914,4.914,6.0,11.650,11.650,30.706,7.0,1.910,1.910,5.0,11.616,11.616,41.123,1.0,0.000,-0.034,5.0,11.784,11.784,29.837,2.0,0.168,0.168,14.744444,0.000000,1.0


# Points

Neither data set contains points awarded so we use a reference file and merge

Merge type here must be `outer` so people that finished outside the top 60, or DNF, don't get trimmed from the dataset

In [87]:
dfp = pd.read_csv( event + '_points_' + ( 'race_' if False == quali else 'qual_' ) + gender + '.csv', index_col=0 )
dfp = dfp.reset_index(drop=False)
df = df.merge( dfp, left_index=True, right_index=True, how="outer")

In [88]:
df.tail()

,bib,id,name,rank,speed,split1,split2,split3,split4,split5,start,start_rev,status,uci,split1_rank,split1_vs_best,split1_vs_winner,split2_rank,split2_vs_best,split2_vs_winner,split2_sector,split2_sector_rank,split2_sector_vs_best,split2_sector_vs_winner,split3_rank,split3_vs_best,split3_vs_winner,split3_sector,split3_sector_rank,split3_sector_vs_best,split3_sector_vs_winner,split4_rank,split4_vs_best,split4_vs_winner,split4_sector,split4_sector_rank,split4_sector_vs_best,split4_sector_vs_winner,split5_rank,split5_vs_best,split5_vs_winner,split5_sector,split5_sector_rank,split5_sector_vs_best,split5_sector_vs_winner,speed_ms,speed_ms_vs_best,speed_rank,points
11,6,2006,HRASTNIK Monika,12,49.25,70.924,228.648,259.526,301.992,334.228,10,7,Finished,10017602955,15.0,6.110,6.110,12.0,17.310,17.310,157.724,12.0,11.200,11.200,11.0,19.392,19.392,30.878,8.0,2.082,2.082,11.0,20.701,20.701,42.466,8.0,1.343,1.309,12.0,23.268,23.268,32.236,14.0,2.567,2.567,13.680556,1.063889,13.0,35.0
12,19,2019,A'HERN Sian,13,47.26,72.004,227.763,261.969,304.970,338.471,6,11,Finished,10010802043,16.0,7.190,7.190,11.0,16.425,16.425,155.759,11.0,9.235,9.235,13.0,21.835,21.835,34.206,15.0,5.410,5.410,13.0,23.679,23.679,43.001,10.0,1.878,1.844,13.0,27.511,27.511,33.501,16.0,3.832,3.832,13.127778,1.616667,15.0,25.0
13,11,2011,MILLER Miranda,14,51.04,70.044,233.287,265.325,308.498,338.761,2,15,Finished,10005872726,11.0,5.230,5.230,14.0,21.949,21.949,163.243,14.0,16.719,16.719,14.0,25.191,25.191,32.038,12.0,3.242,3.242,14.0,27.207,27.207,43.173,12.0,2.050,2.016,14.0,27.801,27.801,30.263,8.0,0.594,0.594,14.177778,0.566667,7.0,15.0
14,18,2018,RONNING Frida Helena,15,51.86,69.709,236.186,279.291,323.358,353.508,3,14,Finished,10008084326,9.0,4.895,4.895,15.0,24.848,24.848,166.477,15.0,19.953,19.953,15.0,39.157,39.157,43.105,16.0,14.309,14.309,15.0,42.067,42.067,44.067,16.0,2.944,2.910,15.0,42.548,42.548,30.150,6.0,0.481,0.481,14.405556,0.338889,4.0,5.0
15,21,2021,MOLLOY Tegan,16,52.84,70.599,250.404,282.587,324.682,354.699,1,16,Finished,10008826980,13.0,5.785,5.785,16.0,39.066,39.066,179.805,16.0,33.281,33.281,16.0,42.453,42.453,32.183,14.0,3.387,3.387,16.0,43.391,43.391,42.095,5.0,0.972,0.938,16.0,43.739,43.739,30.017,4.0,0.348,0.348,14.677778,0.066667,2.0,NaN


# Data Export

All that's left is to save our data to CSV files so we can quickly import it again for analysis and visualization without making constant requests to the online servers. This not only reduces load on the services providing the data, but also allows us to work on our analysis "offline", moreover giving us a local copy in case the results are ever taken down. It's also much quicker to load data this way than constantly hitting online servers.

In [89]:
df.id = df.id.astype(str)
dfm = df.merge( df2, left_on='id', right_index=True, how='inner' )

In [90]:
df.to_csv( file_prefix + '.results.csv' )
df2.to_csv( file_prefix + '.racers.csv' )
dfm.to_csv( file_prefix + '.merged.csv' )

--- 

## Credits

### Author: Dominic Wrapson


> **@domwrap**
<br>
<img src="https://png.icons8.com/material/24/000000/github-2.png">
<img src="https://png.icons8.com/material/24/000000/stackoverflow.png">
<img src="https://png.icons8.com/material/24/000000/linkedin.png">
<img src="https://png.icons8.com/material/24/000000/windows8.png">
<img src="https://png.icons8.com/ios-glyphs/24/000000/instagram-new.png">
<img src="https://png.icons8.com/material/24/000000/twitter.png">
<a href="https://medium.com/@domwrap"><img src="https://png.icons8.com/material/24/000000/medium-logo.png"></a>
>
> <img src="https://png.icons8.com/material/24/000000/home.png"> http://domwrap.me
>
><img src="https://png.icons8.com/material/24/000000/cycling-mountain-bike.png"> [Hwulex](https://www.pinkbike.com/u/Hwulex/)


---

#### Special Thanks

Mark Shilton for the inspiration
- http://lookatthestats.blogspot.ca
- https://plus.google.com/+MarkShilton
- https://dirtmountainbike.com/author/mrgeekstats


<a href="https://icons8.com">Icon pack by Icons8</a>